In [1]:
import sys
import paths
import devito_parser
import ccode_generator
import template_renderer
from devito import Grid, TimeFunction, Eq, Operator, solve

In [2]:
# Define problem: second order wave equation
nt = 100  # Number of timesteps
dt = 0.2 * 2. / 20  # Timestep size (sigma=0.2)
c = 1  # Value for c

In [3]:
grid = Grid(shape=(21, 21), extent=(1., 1.))
u = TimeFunction(name='u', grid=grid, space_order=2, time_order=2)
u

u(t, x, y)

In [4]:
pde = (1 / c*c) * u.dt2 - u.laplace
pde

-Derivative(u(t, x, y), (x, 2)) - Derivative(u(t, x, y), (y, 2)) + 1.0*Derivative(u(t, x, y), (t, 2))

In [5]:
stencil = Eq(u.forward, solve(pde, u.forward))
stencil

Eq(u(t + dt, x, y), 1.0*dt**2*(-2.0*u(t, x, y)/h_y**2 + u(t, x, y - h_y)/h_y**2 + u(t, x, y + h_y)/h_y**2 - 2.0*u(t, x, y)/h_x**2 + u(t, x - h_x, y)/h_x**2 + u(t, x + h_x, y)/h_x**2 + 2.0*u(t, x, y)/dt**2 - 1.0*u(t - dt, x, y)/dt**2))

In [6]:
# Initialize problem data (initial condition)
from examples.cfd import init_smooth, plot_field
init_smooth(field=u.data[0], dx=grid.spacing[0], dy=grid.spacing[1])
init_smooth(field=u.data[1], dx=grid.spacing[0], dy=grid.spacing[1])

In [7]:
# Tranlate DSL
parser = devito_parser.Parser(stencil)
sympy_stencil = parser.translate_expression()
sympy_stencil

Eq(Element(u, indices=(t + 1, x, y)), 1.0*dt**2*(-2.0*Element(u, indices=(t, x, y))/h_y**2 + Element(u, indices=(t, x, y - 1))/h_y**2 + Element(u, indices=(t, x, y + 1))/h_y**2 - 2.0*Element(u, indices=(t, x, y))/h_x**2 + Element(u, indices=(t, x - 1, y))/h_x**2 + Element(u, indices=(t, x + 1, y))/h_x**2 + 2.0*Element(u, indices=(t, x, y))/dt**2 - 1.0*Element(u, indices=(t - 1, x, y))/dt**2))

In [8]:
# Generate solver code
generator = ccode_generator.Generator(sympy_stencil, len(u.grid.dimensions))
code = generator.generate_solver()
code

'int Solver(double dt, double h_x, double h_y, double u[3][21][21], int time_M, int time_m, int x_M, int x_m, int y_M, int y_m){\n   int t;\n   int x;\n   int y;\n   for (t = time_m; t < time_M; t += 1) {\n      for (x = x_m; x < x_M; x += 1) {\n         for (y = y_m; y < y_M; y += 1) {\n            u[(t + 1)%3][x][y] = 1.0*pow(dt, 2)*(-2.0*u[(t)%3][x][y]/pow(h_y, 2) + u[(t)%3][x][y - 1]/pow(h_y, 2) + u[(t)%3][x][y + 1]/pow(h_y, 2) - 2.0*u[(t)%3][x][y]/pow(h_x, 2) + u[(t)%3][x - 1][y]/pow(h_x, 2) + u[(t)%3][x + 1][y]/pow(h_x, 2) + 2.0*u[(t)%3][x][y]/pow(dt, 2) - 1.0*u[(t - 1)%3][x][y]/pow(dt, 2));\n         };\n      };\n   };\n}'

In [9]:
# Render template
renderer = template_renderer.Renderer(u, dt, nt, code)
renderer.render_template()